In [2]:
import os
import shutil

# Define the base directories for test, train, and val
base_dirs = [
    "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/data/gender/test",
    "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/data/gender/train",
    "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/data/gender/val"
]

# Define the keywords for upper and lower body clothing
upper_body_keywords = ["Blouses_Shirts", "Jackets_Vests", "T_Shirts"]
lower_body_keywords = ["Pants", "Shorts", "Skirts"]

# Function to categorize and copy images
def categorize_images(source_dir, upper_body_dir, lower_body_dir):
    for root, _, files in os.walk(source_dir):
        for filename in files:
            if not filename.endswith((".jpg", ".jpeg", ".png")):
                continue

            source_path = os.path.join(root, filename)

            if any(keyword in filename for keyword in upper_body_keywords):
                destination_path = os.path.join(upper_body_dir, filename)
                shutil.copy(source_path, destination_path)
            elif any(keyword in filename for keyword in lower_body_keywords):
                destination_path = os.path.join(lower_body_dir, filename)
                shutil.copy(source_path, destination_path)

# Process each base directory
for base_dir in base_dirs:
    male_dir = os.path.join(base_dir, "male")
    female_dir = os.path.join(base_dir, "female")

    # Define the destination directories for upper and lower body clothing
    upper_body_male_dir = os.path.join(base_dir, "upper_body/male")
    lower_body_male_dir = os.path.join(base_dir, "lower_body/male")
    upper_body_female_dir = os.path.join(base_dir, "upper_body/female")
    lower_body_female_dir = os.path.join(base_dir, "lower_body/female")

    # Ensure the destination directories exist
    os.makedirs(upper_body_male_dir, exist_ok=True)
    os.makedirs(lower_body_male_dir, exist_ok=True)
    os.makedirs(upper_body_female_dir, exist_ok=True)
    os.makedirs(lower_body_female_dir, exist_ok=True)

    # Categorize images for male and female
    categorize_images(male_dir, upper_body_male_dir, lower_body_male_dir)
    categorize_images(female_dir, upper_body_female_dir, lower_body_female_dir)

print("Images have been categorized and copied to the respective directories.")


Images have been categorized and copied to the respective directories.


In [ ]:
import os
import torch
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
import numpy as np
from PIL import Image
from tqdm import tqdm

# Load the pre-trained segmentation model and feature extractor
model_name = "mattmdjaga/segformer_b2_clothes"
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(model_name)

# Check if a GPU is available and use it if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def segment_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=[image], return_tensors="pt").to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the segmentation map
    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(logits, size=image.size[::-1], mode='bilinear', align_corners=False)
    segmentation = upsampled_logits.argmax(dim=1).squeeze().cpu().numpy()
    
    return segmentation, image

def create_transparent_image(image, mask):
    # Convert image to RGBA
    image_np = np.array(image)
    image_rgba = Image.fromarray(image_np).convert("RGBA")

    # Set alpha channel to 0 for background
    alpha_channel = np.where(mask == 0, 0, 255).astype(np.uint8)
    image_rgba.putalpha(Image.fromarray(alpha_channel))

    return image_rgba

def process_folder(input_folder, output_folder, labels, image_size=(256, 256)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    filenames = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]
    for filename in tqdm(filenames):
        image_path = os.path.join(input_folder, filename)
        image = Image.open(image_path).convert("RGB").resize(image_size)
        segmentation, image = segment_image(image_path)

        # Create the mask and transparent image for the given labels
        mask = np.isin(segmentation, labels).astype(np.uint8) * 255
        segmented_image = create_transparent_image(image, mask)
        
        # Save the segmented image
        segmented_image.save(os.path.join(output_folder, filename.replace('.jpg', '.png').replace('.jpeg', '.png')))

# Path to your folders
input_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/filtered_images/female_tops/formal"
output_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/segmented_images/female_top"
labels = [3, 4]  # Adjust these labels as necessary

process_folder(input_folder, output_folder, labels)

c:\Users\ARA\Desktop\Outfit-Aura\OUTFIT-AURA-web-application-\outfit\Lib\site-packages\transformers\models\segformer\feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
c:\Users\ARA\Desktop\Outfit-Aura\OUTFIT-AURA-web-application-\outfit\Lib\site-packages\transformers\utils\deprecation.py:165: UserWarning: The following named arguments are not valid for `SegformerFeatureExtractor.__init__` and were ignored: 'feature_extractor_type'
  return func(*args, **kwargs)
100%|██████████| 36/36 [02:11<00:00,  3.66s/it]


In [ ]:
import os
import torch
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
import numpy as np
from PIL import Image
from tqdm import tqdm

# Load the pre-trained segmentation model and feature extractor
model_name = "mattmdjaga/segformer_b2_clothes"
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(model_name)

# Check if a GPU is available and use it if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def segment_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=[image], return_tensors="pt").to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the segmentation map
    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(logits, size=image.size[::-1], mode='bilinear', align_corners=False)
    segmentation = upsampled_logits.argmax(dim=1).squeeze().cpu().numpy()
    
    return segmentation, image

def create_transparent_image(image, mask):
    # Convert image to RGBA
    image_np = np.array(image)
    image_rgba = Image.fromarray(image_np).convert("RGBA")

    # Set alpha channel to 0 for background
    alpha_channel = np.where(mask == 0, 0, 255).astype(np.uint8)
    image_rgba.putalpha(Image.fromarray(alpha_channel))

    return image_rgba

def process_folder(input_folder, output_folder, labels, image_size=(256, 256)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    filenames = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]
    for filename in tqdm(filenames):
        image_path = os.path.join(input_folder, filename)
        image = Image.open(image_path).convert("RGB").resize(image_size)
        segmentation, image = segment_image(image_path)

        # Create the mask and transparent image for the given labels
        mask = np.isin(segmentation, labels).astype(np.uint8) * 255
        segmented_image = create_transparent_image(image, mask)
        
        # Save the segmented image
        segmented_image.save(os.path.join(output_folder, filename.replace('.jpg', '.png').replace('.jpeg', '.png')))

# Path to your folders
input_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/filtered_images/female_tops/informal"
output_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/segmented_images/female_top_informal"
labels = [3, 4]  # Adjust these labels as necessary

process_folder(input_folder, output_folder, labels)

100%|██████████| 5463/5463 [8:09:59<00:00,  5.38s/it]   


In [ ]:
import os
import torch
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
import numpy as np
from PIL import Image
from tqdm import tqdm

# Load the pre-trained segmentation model and feature extractor
model_name = "mattmdjaga/segformer_b2_clothes"
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(model_name)

# Check if a GPU is available and use it if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def segment_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=[image], return_tensors="pt").to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the segmentation map
    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(logits, size=image.size[::-1], mode='bilinear', align_corners=False)
    segmentation = upsampled_logits.argmax(dim=1).squeeze().cpu().numpy()
    
    return segmentation, image

def create_transparent_image(image, mask):
    # Convert image to RGBA
    image_np = np.array(image)
    image_rgba = Image.fromarray(image_np).convert("RGBA")

    # Set alpha channel to 0 for background
    alpha_channel = np.where(mask == 0, 0, 255).astype(np.uint8)
    image_rgba.putalpha(Image.fromarray(alpha_channel))

    return image_rgba

def process_folder(input_folder, output_folder, labels, image_size=(256, 256)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    filenames = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]
    for filename in tqdm(filenames):
        image_path = os.path.join(input_folder, filename)
        image = Image.open(image_path).convert("RGB").resize(image_size)
        segmentation, image = segment_image(image_path)

        # Create the mask and transparent image for the given labels
        mask = np.isin(segmentation, labels).astype(np.uint8) * 255
        segmented_image = create_transparent_image(image, mask)
        
        # Save the segmented image
        segmented_image.save(os.path.join(output_folder, filename.replace('.jpg', '.png').replace('.jpeg', '.png')))

# Path to your folders
input_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/filtered_images/female_bottoms/formal"
output_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/segmented_images/female_bottoms_formal"
labels = [5, 6, 7, 8] # Adjust these labels as necessary

process_folder(input_folder, output_folder, labels)

c:\Users\ARA\Desktop\Outfit-Aura\OUTFIT-AURA-web-application-\outfit\Lib\site-packages\transformers\models\segformer\feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
c:\Users\ARA\Desktop\Outfit-Aura\OUTFIT-AURA-web-application-\outfit\Lib\site-packages\transformers\utils\deprecation.py:165: UserWarning: The following named arguments are not valid for `SegformerFeatureExtractor.__init__` and were ignored: 'feature_extractor_type'
  return func(*args, **kwargs)
100%|██████████| 41/41 [08:28<00:00, 12.41s/it]


In [ ]:
import os
import torch
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
import numpy as np
from PIL import Image
from tqdm import tqdm

# Load the pre-trained segmentation model and feature extractor
model_name = "mattmdjaga/segformer_b2_clothes"
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(model_name)

# Check if a GPU is available and use it if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def segment_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=[image], return_tensors="pt").to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the segmentation map
    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(logits, size=image.size[::-1], mode='bilinear', align_corners=False)
    segmentation = upsampled_logits.argmax(dim=1).squeeze().cpu().numpy()
    
    return segmentation, image

def create_transparent_image(image, mask):
    # Convert image to RGBA
    image_np = np.array(image)
    image_rgba = Image.fromarray(image_np).convert("RGBA")

    # Set alpha channel to 0 for background
    alpha_channel = np.where(mask == 0, 0, 255).astype(np.uint8)
    image_rgba.putalpha(Image.fromarray(alpha_channel))

    return image_rgba

def process_folder(input_folder, output_folder, labels, image_size=(256, 256)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    filenames = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]
    for filename in tqdm(filenames):
        image_path = os.path.join(input_folder, filename)
        image = Image.open(image_path).convert("RGB").resize(image_size)
        segmentation, image = segment_image(image_path)

        # Create the mask and transparent image for the given labels
        mask = np.isin(segmentation, labels).astype(np.uint8) * 255
        segmented_image = create_transparent_image(image, mask)
        
        # Save the segmented image
        segmented_image.save(os.path.join(output_folder, filename.replace('.jpg', '.png').replace('.jpeg', '.png')))

# Path to your folders
input_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/filtered_images/male_formal_tops"
output_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/segmented_images/male_tops_formal"
labels = [3,4] # Adjust these labels as necessary

process_folder(input_folder, output_folder, labels)

c:\Users\ARA\Desktop\Outfit-Aura\OUTFIT-AURA-web-application-\outfit\Lib\site-packages\transformers\models\segformer\feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
c:\Users\ARA\Desktop\Outfit-Aura\OUTFIT-AURA-web-application-\outfit\Lib\site-packages\transformers\utils\deprecation.py:165: UserWarning: The following named arguments are not valid for `SegformerFeatureExtractor.__init__` and were ignored: 'feature_extractor_type'
  return func(*args, **kwargs)
100%|██████████| 868/868 [2:10:10<00:00,  9.00s/it]  


dress segmentation

In [1]:
import os
import torch
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
import numpy as np
from PIL import Image
from tqdm import tqdm

# Load the pre-trained segmentation model and feature extractor
model_name = "mattmdjaga/segformer_b2_clothes"
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(model_name)

# Check if a GPU is available and use it if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def segment_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=[image], return_tensors="pt").to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the segmentation map
    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(logits, size=image.size[::-1], mode='bilinear', align_corners=False)
    segmentation = upsampled_logits.argmax(dim=1).squeeze().cpu().numpy()
    
    return segmentation, image

def create_transparent_image(image, mask):
    # Convert image to RGBA
    image_np = np.array(image)
    image_rgba = Image.fromarray(image_np).convert("RGBA")

    # Set alpha channel to 0 for background
    alpha_channel = np.where(mask == 0, 0, 255).astype(np.uint8)
    image_rgba.putalpha(Image.fromarray(alpha_channel))

    return image_rgba

def process_folder(input_folder, output_folder, labels, image_size=(256, 256)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    filenames = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]
    for filename in tqdm(filenames):
        image_path = os.path.join(input_folder, filename)
        image = Image.open(image_path).convert("RGB").resize(image_size)
        segmentation, image = segment_image(image_path)

        # Create the mask and transparent image for the given labels
        mask = np.isin(segmentation, labels).astype(np.uint8) * 255
        segmented_image = create_transparent_image(image, mask)
        
        # Save the segmented image
        segmented_image.save(os.path.join(output_folder, filename.replace('.jpg', '.png').replace('.jpeg', '.png')))

# Path to your folders
input_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/filtered_images/male_bottoms/formal"
output_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/segmented_images/male_bottom_formal"
labels = [5, 6, 7, 8] # Adjust these labels as necessary

process_folder(input_folder, output_folder, labels)

c:\Users\ARA\Desktop\Outfit-Aura\OUTFIT-AURA-web-application-\outfit\Lib\site-packages\transformers\models\segformer\feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
c:\Users\ARA\Desktop\Outfit-Aura\OUTFIT-AURA-web-application-\outfit\Lib\site-packages\transformers\utils\deprecation.py:165: UserWarning: The following named arguments are not valid for `SegformerFeatureExtractor.__init__` and were ignored: 'feature_extractor_type'
  return func(*args, **kwargs)
100%|██████████| 207/207 [33:48<00:00,  9.80s/it]


In [8]:
import os
import torch
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
import numpy as np
from PIL import Image
from tqdm import tqdm

# Load the pre-trained segmentation model and feature extractor
model_name = "mattmdjaga/segformer_b2_clothes"
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(model_name)

# Check if a GPU is available and use it if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def segment_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=[image], return_tensors="pt").to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the segmentation map
    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(logits, size=image.size[::-1], mode='bilinear', align_corners=False)
    segmentation = upsampled_logits.argmax(dim=1).squeeze().cpu().numpy()
    
    return segmentation, image

def create_transparent_image(image, mask):
    # Convert image to RGBA
    image_np = np.array(image)
    image_rgba = Image.fromarray(image_np).convert("RGBA")

    # Set alpha channel to 0 for background
    alpha_channel = np.where(mask == 0, 0, 255).astype(np.uint8)
    image_rgba.putalpha(Image.fromarray(alpha_channel))

    return image_rgba

def process_folder(input_folder, output_folder, image_size=(256, 256)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    filenames = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    for filename in tqdm(filenames):
        image_path = os.path.join(input_folder, filename)
        segmentation, image = segment_image(image_path)

        # Create the mask and transparent image
        mask = (segmentation != 0).astype(np.uint8) * 255
        segmented_image = create_transparent_image(image, mask)
        
        # Save the segmented image
        segmented_image.save(os.path.join(output_folder, filename.replace('.jpg', '.png').replace('.jpeg', '.png')))

# Path to your folders
input_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/zalando/test/cloth"
output_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/segmented_images/female_tops_informal"

process_folder(input_folder, output_folder)


In [10]:
import os
import torch
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
import numpy as np
from PIL import Image
from tqdm import tqdm

# Load the pre-trained segmentation model and feature extractor
model_name = "mattmdjaga/segformer_b2_clothes"
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(model_name)

# Check if a GPU is available and use it if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def segment_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=[image], return_tensors="pt").to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the segmentation map
    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(logits, size=image.size[::-1], mode='bilinear', align_corners=False)
    segmentation = upsampled_logits.argmax(dim=1).squeeze().cpu().numpy()
    
    return segmentation, image

def create_transparent_image(image, mask):
    # Convert image to RGBA
    image_np = np.array(image)
    image_rgba = Image.fromarray(image_np).convert("RGBA")

    # Set alpha channel to 0 for background
    alpha_channel = np.where(mask == 0, 0, 255).astype(np.uint8)
    image_rgba.putalpha(Image.fromarray(alpha_channel))

    return image_rgba

def process_folder(input_folder, output_folder, image_size=(256, 256)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    filenames = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    for filename in tqdm(filenames, desc="Processing images"):
        image_path = os.path.join(input_folder, filename)
        segmentation, image = segment_image(image_path)

        # Create the mask and transparent image
        mask = (segmentation != 0).astype(np.uint8) * 255
        segmented_image = create_transparent_image(image, mask)
        
        # Save the segmented image
        segmented_image.save(os.path.join(output_folder, filename.replace('.jpg', '.png').replace('.jpeg', '.png')))

# Path to your folders
input_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/dress/black-dress/test"  # Change to your input folder path
output_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/segmented_images/female_dresses"  # Change to your output folder path

process_folder(input_folder, output_folder)















































Processing images: 100%|██████████| 44/44 [04:22<00:00,  5.97s/it]


In [11]:
import os
import torch
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
import numpy as np
from PIL import Image
from tqdm import tqdm

# Load the pre-trained segmentation model and feature extractor
model_name = "mattmdjaga/segformer_b2_clothes"
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(model_name)

# Check if a GPU is available and use it if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def segment_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=[image], return_tensors="pt").to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the segmentation map
    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(logits, size=image.size[::-1], mode='bilinear', align_corners=False)
    segmentation = upsampled_logits.argmax(dim=1).squeeze().cpu().numpy()
    
    return segmentation, image

def create_transparent_image(image, mask):
    # Convert image to RGBA
    image_np = np.array(image)
    image_rgba = Image.fromarray(image_np).convert("RGBA")

    # Set alpha channel to 0 for background
    alpha_channel = np.where(mask == 0, 0, 255).astype(np.uint8)
    image_rgba.putalpha(Image.fromarray(alpha_channel))

    return image_rgba

def process_folder(input_folder, output_folder, image_size=(256, 256)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    filenames = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    for filename in tqdm(filenames, desc="Processing images"):
        image_path = os.path.join(input_folder, filename)
        segmentation, image = segment_image(image_path)

        # Create the mask and transparent image
        mask = (segmentation != 0).astype(np.uint8) * 255
        segmented_image = create_transparent_image(image, mask)
        
        # Save the segmented image
        segmented_image.save(os.path.join(output_folder, filename.replace('.jpg', '.png').replace('.jpeg', '.png')))

# Path to your folders
input_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/dress/red-dress/test"  # Change to your input folder path
output_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/segmented_images/female_dresses/red"  # Change to your output folder path

process_folder(input_folder, output_folder)








































































































Processing images: 100%|██████████| 101/101 [11:22<00:00,  6.76s/it]


In [12]:
import os
import torch
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
import numpy as np
from PIL import Image
from tqdm import tqdm

# Load the pre-trained segmentation model and feature extractor
model_name = "mattmdjaga/segformer_b2_clothes"
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(model_name)

# Check if a GPU is available and use it if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def segment_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=[image], return_tensors="pt").to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the segmentation map
    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(logits, size=image.size[::-1], mode='bilinear', align_corners=False)
    segmentation = upsampled_logits.argmax(dim=1).squeeze().cpu().numpy()
    
    return segmentation, image

def create_transparent_image(image, mask):
    # Convert image to RGBA
    image_np = np.array(image)
    image_rgba = Image.fromarray(image_np).convert("RGBA")

    # Set alpha channel to 0 for background
    alpha_channel = np.where(mask == 0, 0, 255).astype(np.uint8)
    image_rgba.putalpha(Image.fromarray(alpha_channel))

    return image_rgba

def process_folder(input_folder, output_folder, image_size=(256, 256)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    filenames = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    for filename in tqdm(filenames, desc="Processing images"):
        image_path = os.path.join(input_folder, filename)
        segmentation, image = segment_image(image_path)

        # Create the mask and transparent image
        mask = (segmentation != 0).astype(np.uint8) * 255
        segmented_image = create_transparent_image(image, mask)
        
        # Save the segmented image
        segmented_image.save(os.path.join(output_folder, filename.replace('.jpg', '.png').replace('.jpeg', '.png')))

# Path to your folders
input_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/dress/blue-dress/test"  # Change to your input folder path
output_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/segmented_images/female_dresses/blue"  # Change to your output folder path

process_folder(input_folder, output_folder)











































































































































Processing images: 100%|██████████| 136/136 [17:19<00:00,  7.65s/it]


In [21]:
import os
import torch
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
import numpy as np
from PIL import Image
from tqdm import tqdm

# Load the pre-trained segmentation model and feature extractor
model_name = "mattmdjaga/segformer_b2_clothes"
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(model_name)

# Check if a GPU is available and use it if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def segment_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=[image], return_tensors="pt").to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the segmentation map
    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(logits, size=image.size[::-1], mode='bilinear', align_corners=False)
    segmentation = upsampled_logits.argmax(dim=1).squeeze().cpu().numpy()
    
    return segmentation, image

def create_transparent_image(image, mask):
    # Convert image to RGBA
    image_np = np.array(image)
    image_rgba = Image.fromarray(image_np).convert("RGBA")

    # Set alpha channel to 0 for background
    alpha_channel = np.where(mask == 0, 0, 255).astype(np.uint8)
    image_rgba.putalpha(Image.fromarray(alpha_channel))

    return image_rgba

def process_folder(input_folder, output_folder, image_size=(256, 256)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    filenames = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    for filename in tqdm(filenames, desc="Processing images"):
        image_path = os.path.join(input_folder, filename)
        segmentation, image = segment_image(image_path)

        # Create the mask and transparent image
        mask = (segmentation != 0).astype(np.uint8) * 255
        segmented_image = create_transparent_image(image, mask)
        
        # Save the segmented image
        segmented_image.save(os.path.join(output_folder, filename.replace('.jpg', '.png').replace('.jpeg', '.png')))

# Path to your folders
input_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/dress/white-dress/test"  # Change to your input folder path
output_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/segmented_images/female_dresses/white"  # Change to your output folder path

process_folder(input_folder, output_folder)





























































































Processing images: 100%|██████████| 90/90 [07:36<00:00,  5.07s/it]


In [13]:
import os
import torch
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
import numpy as np
from PIL import Image
from tqdm import tqdm

# Load the pre-trained segmentation model and feature extractor
model_name = "mattmdjaga/segformer_b2_clothes"
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(model_name)

# Check if a GPU is available and use it if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def segment_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=[image], return_tensors="pt").to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the segmentation map
    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(logits, size=image.size[::-1], mode='bilinear', align_corners=False)
    segmentation = upsampled_logits.argmax(dim=1).squeeze().cpu().numpy()
    
    return segmentation, image

def create_transparent_image(image, mask):
    # Convert image to RGBA
    image_np = np.array(image)
    image_rgba = Image.fromarray(image_np).convert("RGBA")

    # Set alpha channel to 0 for background
    alpha_channel = np.where(mask == 0, 0, 255).astype(np.uint8)
    image_rgba.putalpha(Image.fromarray(alpha_channel))

    return image_rgba

def process_folder(input_folder, output_folder, image_size=(256, 256)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    filenames = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    for filename in tqdm(filenames, desc="Processing images"):
        image_path = os.path.join(input_folder, filename)
        segmentation, image = segment_image(image_path)

        # Create the mask and transparent image
        mask = (segmentation != 0).astype(np.uint8) * 255
        segmented_image = create_transparent_image(image, mask)
        
        # Save the segmented image
        segmented_image.save(os.path.join(output_folder, filename.replace('.jpg', '.png').replace('.jpeg', '.png')))

# Path to your folders
input_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/dress/black-dress/train"  # Change to your input folder path
output_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/segmented_images/female_dresses/black"  # Change to your output folder path

process_folder(input_folder, output_folder)


























































































































































































































































Processing images: 100%|██████████| 247/247 [22:23<00:00,  5.44s/it]


In [14]:
import os
import torch
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
import numpy as np
from PIL import Image
from tqdm import tqdm

# Load the pre-trained segmentation model and feature extractor
model_name = "mattmdjaga/segformer_b2_clothes"
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(model_name)

# Check if a GPU is available and use it if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def segment_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=[image], return_tensors="pt").to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the segmentation map
    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(logits, size=image.size[::-1], mode='bilinear', align_corners=False)
    segmentation = upsampled_logits.argmax(dim=1).squeeze().cpu().numpy()
    
    return segmentation, image

def create_transparent_image(image, mask):
    # Convert image to RGBA
    image_np = np.array(image)
    image_rgba = Image.fromarray(image_np).convert("RGBA")

    # Set alpha channel to 0 for background
    alpha_channel = np.where(mask == 0, 0, 255).astype(np.uint8)
    image_rgba.putalpha(Image.fromarray(alpha_channel))

    return image_rgba

def process_folder(input_folder, output_folder, image_size=(256, 256)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    filenames = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    for filename in tqdm(filenames, desc="Processing images"):
        image_path = os.path.join(input_folder, filename)
        segmentation, image = segment_image(image_path)

        # Create the mask and transparent image
        mask = (segmentation != 0).astype(np.uint8) * 255
        segmented_image = create_transparent_image(image, mask)
        
        # Save the segmented image
        segmented_image.save(os.path.join(output_folder, filename.replace('.jpg', '.png').replace('.jpeg', '.png')))

# Path to your folders
input_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/dress/black-dress/val"  # Change to your input folder path
output_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/segmented_images/female_dresses/black"  # Change to your output folder path

process_folder(input_folder, output_folder)












































































































































Processing images: 100%|██████████| 137/137 [11:22<00:00,  4.98s/it]


In [15]:
import os
import torch
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
import numpy as np
from PIL import Image
from tqdm import tqdm

# Load the pre-trained segmentation model and feature extractor
model_name = "mattmdjaga/segformer_b2_clothes"
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(model_name)

# Check if a GPU is available and use it if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def segment_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=[image], return_tensors="pt").to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the segmentation map
    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(logits, size=image.size[::-1], mode='bilinear', align_corners=False)
    segmentation = upsampled_logits.argmax(dim=1).squeeze().cpu().numpy()
    
    return segmentation, image

def create_transparent_image(image, mask):
    # Convert image to RGBA
    image_np = np.array(image)
    image_rgba = Image.fromarray(image_np).convert("RGBA")

    # Set alpha channel to 0 for background
    alpha_channel = np.where(mask == 0, 0, 255).astype(np.uint8)
    image_rgba.putalpha(Image.fromarray(alpha_channel))

    return image_rgba

def process_folder(input_folder, output_folder, image_size=(256, 256)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    filenames = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    for filename in tqdm(filenames, desc="Processing images"):
        image_path = os.path.join(input_folder, filename)
        segmentation, image = segment_image(image_path)

        # Create the mask and transparent image
        mask = (segmentation != 0).astype(np.uint8) * 255
        segmented_image = create_transparent_image(image, mask)
        
        # Save the segmented image
        segmented_image.save(os.path.join(output_folder, filename.replace('.jpg', '.png').replace('.jpeg', '.png')))

# Path to your folders
input_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/dress/blue-dress/train"  # Change to your input folder path
output_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/segmented_images/female_dresses/blue"  # Change to your output folder path

process_folder(input_folder, output_folder)













































































































































































































































































































Processing images: 100%|██████████| 298/298 [25:48<00:00,  5.20s/it]


In [16]:
import os
import torch
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
import numpy as np
from PIL import Image
from tqdm import tqdm

# Load the pre-trained segmentation model and feature extractor
model_name = "mattmdjaga/segformer_b2_clothes"
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(model_name)

# Check if a GPU is available and use it if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def segment_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=[image], return_tensors="pt").to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the segmentation map
    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(logits, size=image.size[::-1], mode='bilinear', align_corners=False)
    segmentation = upsampled_logits.argmax(dim=1).squeeze().cpu().numpy()
    
    return segmentation, image

def create_transparent_image(image, mask):
    # Convert image to RGBA
    image_np = np.array(image)
    image_rgba = Image.fromarray(image_np).convert("RGBA")

    # Set alpha channel to 0 for background
    alpha_channel = np.where(mask == 0, 0, 255).astype(np.uint8)
    image_rgba.putalpha(Image.fromarray(alpha_channel))

    return image_rgba

def process_folder(input_folder, output_folder, image_size=(256, 256)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    filenames = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    for filename in tqdm(filenames, desc="Processing images"):
        image_path = os.path.join(input_folder, filename)
        segmentation, image = segment_image(image_path)

        # Create the mask and transparent image
        mask = (segmentation != 0).astype(np.uint8) * 255
        segmented_image = create_transparent_image(image, mask)
        
        # Save the segmented image
        segmented_image.save(os.path.join(output_folder, filename.replace('.jpg', '.png').replace('.jpeg', '.png')))

# Path to your folders
input_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/dress/blue-dress/val"  # Change to your input folder path
output_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/segmented_images/female_dresses/blue"  # Change to your output folder path

process_folder(input_folder, output_folder)
















































Processing images: 100%|██████████| 45/45 [03:50<00:00,  5.11s/it]


In [17]:
import os
import torch
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
import numpy as np
from PIL import Image
from tqdm import tqdm

# Load the pre-trained segmentation model and feature extractor
model_name = "mattmdjaga/segformer_b2_clothes"
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(model_name)

# Check if a GPU is available and use it if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def segment_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=[image], return_tensors="pt").to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the segmentation map
    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(logits, size=image.size[::-1], mode='bilinear', align_corners=False)
    segmentation = upsampled_logits.argmax(dim=1).squeeze().cpu().numpy()
    
    return segmentation, image

def create_transparent_image(image, mask):
    # Convert image to RGBA
    image_np = np.array(image)
    image_rgba = Image.fromarray(image_np).convert("RGBA")

    # Set alpha channel to 0 for background
    alpha_channel = np.where(mask == 0, 0, 255).astype(np.uint8)
    image_rgba.putalpha(Image.fromarray(alpha_channel))

    return image_rgba

def process_folder(input_folder, output_folder, image_size=(256, 256)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    filenames = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    for filename in tqdm(filenames, desc="Processing images"):
        image_path = os.path.join(input_folder, filename)
        segmentation, image = segment_image(image_path)

        # Create the mask and transparent image
        mask = (segmentation != 0).astype(np.uint8) * 255
        segmented_image = create_transparent_image(image, mask)
        
        # Save the segmented image
        segmented_image.save(os.path.join(output_folder, filename.replace('.jpg', '.png').replace('.jpeg', '.png')))

# Path to your folders
input_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/dress/red-dress/train"  # Change to your input folder path
output_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/segmented_images/female_dresses/red"  # Change to your output folder path

process_folder(input_folder, output_folder)



















































































































































































































































































































































































































































Processing images: 100%|██████████| 432/432 [38:35<00:00,  5.36s/it]


In [18]:
import os
import torch
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
import numpy as np
from PIL import Image
from tqdm import tqdm

# Load the pre-trained segmentation model and feature extractor
model_name = "mattmdjaga/segformer_b2_clothes"
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(model_name)

# Check if a GPU is available and use it if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def segment_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=[image], return_tensors="pt").to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the segmentation map
    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(logits, size=image.size[::-1], mode='bilinear', align_corners=False)
    segmentation = upsampled_logits.argmax(dim=1).squeeze().cpu().numpy()
    
    return segmentation, image

def create_transparent_image(image, mask):
    # Convert image to RGBA
    image_np = np.array(image)
    image_rgba = Image.fromarray(image_np).convert("RGBA")

    # Set alpha channel to 0 for background
    alpha_channel = np.where(mask == 0, 0, 255).astype(np.uint8)
    image_rgba.putalpha(Image.fromarray(alpha_channel))

    return image_rgba

def process_folder(input_folder, output_folder, image_size=(256, 256)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    filenames = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    for filename in tqdm(filenames, desc="Processing images"):
        image_path = os.path.join(input_folder, filename)
        segmentation, image = segment_image(image_path)

        # Create the mask and transparent image
        mask = (segmentation != 0).astype(np.uint8) * 255
        segmented_image = create_transparent_image(image, mask)
        
        # Save the segmented image
        segmented_image.save(os.path.join(output_folder, filename.replace('.jpg', '.png').replace('.jpeg', '.png')))

# Path to your folders
input_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/dress/red-dress/val"  # Change to your input folder path
output_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/segmented_images/female_dresses/red"  # Change to your output folder path

process_folder(input_folder, output_folder)















































































































































































































































Processing images: 100%|██████████| 236/236 [20:19<00:00,  5.17s/it]


In [19]:
import os
import torch
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
import numpy as np
from PIL import Image
from tqdm import tqdm

# Load the pre-trained segmentation model and feature extractor
model_name = "mattmdjaga/segformer_b2_clothes"
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(model_name)

# Check if a GPU is available and use it if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def segment_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=[image], return_tensors="pt").to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the segmentation map
    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(logits, size=image.size[::-1], mode='bilinear', align_corners=False)
    segmentation = upsampled_logits.argmax(dim=1).squeeze().cpu().numpy()
    
    return segmentation, image

def create_transparent_image(image, mask):
    # Convert image to RGBA
    image_np = np.array(image)
    image_rgba = Image.fromarray(image_np).convert("RGBA")

    # Set alpha channel to 0 for background
    alpha_channel = np.where(mask == 0, 0, 255).astype(np.uint8)
    image_rgba.putalpha(Image.fromarray(alpha_channel))

    return image_rgba

def process_folder(input_folder, output_folder, image_size=(256, 256)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    filenames = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    for filename in tqdm(filenames, desc="Processing images"):
        image_path = os.path.join(input_folder, filename)
        segmentation, image = segment_image(image_path)

        # Create the mask and transparent image
        mask = (segmentation != 0).astype(np.uint8) * 255
        segmented_image = create_transparent_image(image, mask)
        
        # Save the segmented image
        segmented_image.save(os.path.join(output_folder, filename.replace('.jpg', '.png').replace('.jpeg', '.png')))

# Path to your folders
input_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/dress/white-dress/train"  # Change to your input folder path
output_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/segmented_images/female_dresses/white"  # Change to your output folder path

process_folder(input_folder, output_folder)




























































































































































































































































































































































































































































































































































































































































































































































































Processing images: 100%|██████████| 761/761 [1:06:50<00:00,  5.27s/it]


In [20]:
import os
import torch
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
import numpy as np
from PIL import Image
from tqdm import tqdm

# Load the pre-trained segmentation model and feature extractor
model_name = "mattmdjaga/segformer_b2_clothes"
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(model_name)

# Check if a GPU is available and use it if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def segment_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=[image], return_tensors="pt").to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the segmentation map
    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(logits, size=image.size[::-1], mode='bilinear', align_corners=False)
    segmentation = upsampled_logits.argmax(dim=1).squeeze().cpu().numpy()
    
    return segmentation, image

def create_transparent_image(image, mask):
    # Convert image to RGBA
    image_np = np.array(image)
    image_rgba = Image.fromarray(image_np).convert("RGBA")

    # Set alpha channel to 0 for background
    alpha_channel = np.where(mask == 0, 0, 255).astype(np.uint8)
    image_rgba.putalpha(Image.fromarray(alpha_channel))

    return image_rgba

def process_folder(input_folder, output_folder, image_size=(256, 256)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    filenames = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    for filename in tqdm(filenames, desc="Processing images"):
        image_path = os.path.join(input_folder, filename)
        segmentation, image = segment_image(image_path)

        # Create the mask and transparent image
        mask = (segmentation != 0).astype(np.uint8) * 255
        segmented_image = create_transparent_image(image, mask)
        
        # Save the segmented image
        segmented_image.save(os.path.join(output_folder, filename.replace('.jpg', '.png').replace('.jpeg', '.png')))

# Path to your folders
input_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/dress/white-dress/val"  # Change to your input folder path
output_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/segmented_images/female_dresses/white"  # Change to your output folder path

process_folder(input_folder, output_folder)























































































































































































































































Processing images: 100%|██████████| 244/244 [20:32<00:00,  5.05s/it]


In [1]:
import os
import torch
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
import numpy as np
from PIL import Image
from tqdm import tqdm

# Load the pre-trained segmentation model and feature extractor
model_name = "mattmdjaga/segformer_b2_clothes"
feature_extractor = SegformerFeatureExtractor.from_pretrained(model_name)
model = SegformerForSemanticSegmentation.from_pretrained(model_name)

# Check if a GPU is available and use it if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def segment_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=[image], return_tensors="pt").to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the segmentation map
    logits = outputs.logits
    upsampled_logits = torch.nn.functional.interpolate(logits, size=image.size[::-1], mode='bilinear', align_corners=False)
    segmentation = upsampled_logits.argmax(dim=1).squeeze().cpu().numpy()
    
    return segmentation, image

def create_transparent_image(image, mask):
    # Convert image to RGBA
    image_np = np.array(image)
    image_rgba = Image.fromarray(image_np).convert("RGBA")

    # Set alpha channel to 0 for background
    alpha_channel = np.where(mask == 0, 0, 255).astype(np.uint8)
    image_rgba.putalpha(Image.fromarray(alpha_channel))

    return image_rgba

def process_folder(input_folder, output_folder, image_size=(256, 256)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    filenames = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    for filename in tqdm(filenames):
        image_path = os.path.join(input_folder, filename)
        segmentation, image = segment_image(image_path)

        # Create the mask and transparent image
        mask = (segmentation != 0).astype(np.uint8) * 255
        segmented_image = create_transparent_image(image, mask)
        
        # Save the segmented image with a consistent file extension
        new_filename = os.path.splitext(filename)[0] + '.png'
        segmented_image.save(os.path.join(output_folder, new_filename))

# Path to your folders
input_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/zalando/train/cloth"
output_folder = "C:/Users/ARA/Desktop/Outfit-Aura/OUTFIT-AURA-web-application-/backend/data/segmented_images/informal_tops_female"

process_folder(input_folder, output_folder)


c:\Users\ARA\Desktop\Outfit-Aura\OUTFIT-AURA-web-application-\outfit\Lib\site-packages\transformers\models\segformer\feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
c:\Users\ARA\Desktop\Outfit-Aura\OUTFIT-AURA-web-application-\outfit\Lib\site-packages\transformers\utils\deprecation.py:165: UserWarning: The following named arguments are not valid for `SegformerFeatureExtractor.__init__` and were ignored: 'feature_extractor_type'
  return func(*args, **kwargs)
100%|██████████| 11647/11647 [23:31:57<00:00,  7.27s/it]  
